In [1]:
import pandas as pd
import os
from itertools import product

In [2]:
# Define directories
malware_dir = r"C:\Users\KUNG_LOBSTER69\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_CYBER_SECURITY\RESULT\03.sgFCMed\malware"
benign_dir = r"C:\Users\KUNG_LOBSTER69\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_CYBER_SECURITY\RESULT\03.sgFCMed\benign"

# Define save directory
save_dir = r"C:\Users\KUNG_LOBSTER69\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_CYBER_SECURITY\RESULT\04.COMBINED_TRAIN_DATA"  # Replace with your desired path
os.makedirs(save_dir, exist_ok=True)  # Create the directory if it doesn't exist

# Function to load JSON files with specific naming pattern
def load_json_files_separately(directory):
    data_frames = {}  # Dictionary to store DataFrames
    for filename in os.listdir(directory):
        if filename.startswith("prototypes_") and filename.endswith(".json"):  # Filter files
            file_path = os.path.join(directory, filename)
            try:
                # Load the JSON file into a DataFrame
                df = pd.read_json(file_path)
                # Store DataFrame with filename (without extension) as key
                data_frames[os.path.splitext(filename)[0]] = df
            except Exception as e:
                print(f"Error loading {file_path}: {e}")
    return data_frames

# Load data from both directories
malware_data = load_json_files_separately(malware_dir)
benign_data = load_json_files_separately(benign_dir)

In [3]:
# Combine matching keys from malware_data and benign_data
for malware_key, benign_key in product(malware_data.keys(), benign_data.keys()):
    # Combine DataFrames with additional columns to identify the source and file pair
    combined_df = pd.concat(
        [
            malware_data[malware_key].assign(Source="Malware", Pair=f"{malware_key}-{benign_key}"),
            benign_data[benign_key].assign(Source="Benign", Pair=f"{malware_key}-{benign_key}")
        ],
        ignore_index=True
    )
    
    # Extract the number part from keys
    malware_number = malware_key.replace("prototypes_", "")
    benign_number = benign_key.replace("prototypes_", "")
    
    # Create a descriptive filename
    save_filename = f"malware_{malware_number}-benign_{benign_number}.json"
    save_path = os.path.join(save_dir, save_filename)
    
    # Save the combined DataFrame as a JSON file
    combined_df.to_json(save_path, orient='records')